## 탄소배출권 가격 예측 프로그램

- 사용 데이터 : 종가, 시가, 고가, 저가, 거래량, 거래대금, 변환 날짜, 대비
- 2015년부터 2020년 6월까지의 데이터를 사용하여 모델 생성
- 생성한 모델로 2020년 7월부터 8월까지의 데이터를 테스트, 정확도 비교

### 1. 데이터 임포트

In [7]:
import pandas as pd
kau19 = pd.read_excel('KAU19.xls', names=['date', 'end_pr', 'diff', 'diff_per','start_pr', 'high_pr', 'low_pr', 'volume', 'sum_pr', '_', '__', '___'])
kau20 = pd.read_excel('KAU20.xls', names=['date', 'end_pr', 'diff', 'diff_per','start_pr', 'high_pr', 'low_pr', 'volume', 'sum_pr', '_', '__', '___'])
kau19 = kau19.loc[:, 'date':'sum_pr']
kau20 = kau20.loc[:, 'date':'sum_pr']
print(kau19.head())
print(kau20.head())

         date  end_pr   diff  diff_per start_pr high_pr  low_pr  volume  \
0  2020/08/21  19,850    150      0.76   19,700  19,850  19,700  10,500   
1  2020/08/20  19,700    700      3.68   19,500  19,700  19,300   9,000   
2  2020/08/19  19,000  1,200      6.74   17,900  19,000  17,900  18,200   
3  2020/08/18  17,800     50      0.28   17,750  17,800  17,750  10,000   
4  2020/08/14  17,750     50      0.28   17,500  17,750  17,500   7,400   

        sum_pr  
0  207,825,000  
1  174,500,000  
2  337,875,000  
3  177,625,000  
4  130,765,000  
         date  end_pr    diff  diff_per start_pr high_pr  low_pr  volume  \
0  2020/08/21  21,000   1,450      7.42   21,500  21,500  21,000   1,001   
1  2020/08/20  19,550   1,750      9.83   18,100  19,550  18,100  10,150   
2  2020/08/19  17,800   2,200    -11.00        -       -       -       0   
3  2020/08/18  20,000       0      0.00        -       -       -       0   
4  2020/08/14  20,000  -1,000     -4.76   20,000  20,000  20,000   

## 데이터 분리
- 학습할 데이터와 테스트 데이터 분리
- 2020년 6월 이전까지의 데이터를 사용해서 학습
- 2020년 6월 이후 데이터를 사용해서 테스트